In [4]:
import sys
sys.path.append('../')

import os
import pandas as pd

from glob import glob
from tqdm import tqdm
from collections import defaultdict

from utils import *

In [5]:
data_dir = '/media/jaeho/SSD/datasets/pet_data'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'val')

In [7]:
df_dict = {}
err_cnt = 0

for task in ['train', 'val']:
    print(task)
    json_paths = glob(os.path.join(data_dir, task, '*', 'labels', '*', '*'))
    
    data_dict = defaultdict(list)
    label_cnt = defaultdict(int)
    for json_path in tqdm(json_paths):
        # load image_paths
        target_dir_name = os.path.basename(json_path).replace('.json', '')
        img_dir = json_path.replace("labels", "images").replace(".json", "")

        # open json
        js = open_json(json_path)
        emotion = js['metadata']['inspect']['emotion']

        for anno in js['annotations']:
            frame_num = anno['frame_number']
            time_stamp = anno['timestamp']
            img_path = f"frame_{frame_num}_timestamp_{time_stamp}.jpg"
            img_path = os.path.join(img_dir, img_path)
            if not os.path.exists((img_path)):
                continue

            bbox = convert_bbox_info(anno['bounding_box'])
            int_bbox = list(map(int, bbox))
            if min(int_bbox) < 0:
                err_cnt += 1
                continue
            
            data_dict['image_path'].append(img_path)
            data_dict['bbox'].append(','.join(bbox))
            data_dict['labels'].append(emotion)
            label_cnt[emotion] += 1
    
    df_dict[task] = {
        'dataframe' : pd.DataFrame(data_dict),
        'count' : label_cnt
    }
    print(json.dumps(label_cnt, indent=2, ensure_ascii=False))

train


100%|██████████| 61081/61081 [06:13<00:00, 163.45it/s]


{
  "화남/불쾌": 74578,
  "공포": 4795,
  "행복/즐거움": 515248,
  "편안/안정": 1364794,
  "공격성": 115619,
  "불안/슬픔": 52928
}
val


100%|██████████| 7649/7649 [01:27<00:00, 87.07it/s] 


{
  "편안/안정": 466171,
  "공격성": 16207,
  "공포": 2606,
  "화남/불쾌": 25181,
  "행복/즐거움": 219600,
  "불안/슬픔": 27313
}


In [8]:
err_cnt

124

In [9]:
for task in df_dict:
    df_dict[task]['dataframe'].to_csv(f'../data/{task}.csv', index=False)